In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import moviepy.editor as mpy
from bs4 import BeautifulSoup

In [2]:
def func(lst, str2):
    k = False
    for i in range(len(str2)):
        if str2[i].isalpha():
            k = i
        else:
            break
    if k:
        lst.append(str2[:k+1])         
        str2 = str2[k+1:]
    return lst, str2

In [3]:
str1 = ['Royal', 'Dutch']
str2 = 'Shell175,986.1'
func(str1, str2)

(['Royal', 'Dutch', 'Shell'], '175,986.1')

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_public_corporations_by_market_capitalization'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

In [5]:
tables = soup.find_all('table', class_="wikitable")
tables = tables[1:-2]

In [6]:
df = pd.DataFrame(columns=["Year", "Name", "Capitalisation"])

In [7]:
year = 2021
for table in tables[:15]:
    year -= 1
    for tr in table.find_all('tr')[1:]:
        td = tr.find_all('td')[6]
        td = td.text.split('[')[0]
        td = td.split()
        name, capital = func(td[:-1], td[-1])
        df = df.append({"Year": year, "Name": name[0], "Capitalisation": float(capital.replace(",", ""))}, 
                       ignore_index=True)
        

In [8]:
year = 2005
for table in tables[15:]:
    year -= 1
    for tr in table.find_all('tr')[1:]:
        name = tr.find_all('td')[1].text
        capital = tr.find_all('td')[4].text
        df = df.append({"Year": year, "Name": name, "Capitalisation": float(capital.replace(",", ""))}, 
                       ignore_index=True)

In [9]:
df

,Year,Name,Capitalisation
0,2020,Apple,1981000.0
1,2020,Microsoft,1592000.0
2,2020,Amazon.com,1577000.0
3,2020,Alphabet,999570.0
4,2020,Alibaba,795400.0
...,...,...,...
225,1997,Intel Corporation,150838.0
226,1997,Nippon Telegraph and Telephone,146139.0
227,1997,Merck,120757.0
228,1997,Toyota Motor Corporation,116585.0


In [10]:
for year in range(1997, 2021):
    if year != 2005:
        df_year = df[df["Year"]==year]
        fig, ax = plt.subplots(figsize=(20, 15)) 
        ax.pie(df_year["Capitalisation"], labels=df_year["Name"], autopct='%0.f%%', 
                            shadow=True, startangle=0, pctdistance=0.90, textprops={'fontsize': 18})
        circle = plt.Circle((0,0),0.70,fc='white') 
        donut = plt.gcf()
        donut.gca().add_artist(circle)  
        plt.title("The 10 largest public companies by market capitalisation: 1997-2020",fontsize = 30)
        ax.text(1.5, -1.3, "{} year".format(year), fontsize=40,
            horizontalalignment="right",
            verticalalignment="bottom")
        plt.savefig("{}.png".format(year))
        plt.close()

In [11]:
gif_name = 'donut_chart.gif'
fps = 1
file_list = ["{}.png".format(year) for year in range(1997, 2021) if year != 2005]
clip = mpy.ImageSequenceClip(file_list, fps=fps)
clip.write_gif('{}.gif'.format(gif_name), fps=fps)

MoviePy - Building file donut_chart.gif.gif with imageio.
